In [1]:
# train_fragility.ipynb
import os, numpy as np, tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

DATA_DIR = r"../Datasets/FragilityLevelDetection"
BATCH = 16
IMG = (224,224)

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2,
                                   rotation_range=15, zoom_range=0.1,
                                   width_shift_range=0.1, height_shift_range=0.1,
                                   horizontal_flip=True)
train_gen = train_datagen.flow_from_directory(DATA_DIR, target_size=IMG, batch_size=BATCH,
                                              class_mode='categorical', subset='training')
val_gen = train_datagen.flow_from_directory(DATA_DIR, target_size=IMG, batch_size=BATCH,
                                              class_mode='categorical', subset='validation')

# class weights
labels = train_gen.classes
class_weights = dict(enumerate(compute_class_weight('balanced', classes=np.unique(labels), y=labels)))
print("class_weights:", class_weights)

# base model
base = tf.keras.applications.EfficientNetB0(input_shape=(IMG[0],IMG[1],3), include_top=False, weights='imagenet')
base.trainable = False

x = base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.002))(x)
x = layers.Dropout(0.5)(x)
out = layers.Dense(train_gen.num_classes, activation='softmax')(x)

model = models.Model(inputs=base.input, outputs=out)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# callbacks
os.makedirs("../models", exist_ok=True)
cp = ModelCheckpoint("../Models/fragility_best.keras", monitor='val_accuracy', save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rl = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)

# train
history = model.fit(train_gen, validation_data=val_gen, epochs=30, class_weight=class_weights, callbacks=[cp, es, rl])

# fine-tune
base.trainable = True
for layer in base.layers[:-50]:
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_f = model.fit(train_gen, validation_data=val_gen, epochs=15, class_weight=class_weights, callbacks=[cp, es, rl])

# evaluate on validation/test later


Found 2829 images belonging to 2 classes.
Found 706 images belonging to 2 classes.
class_weights: {0: np.float64(1.5408496732026145), 1: np.float64(0.7401883830455259)}


ValueError: Shape mismatch in layer #1 (named stem_conv)for weight stem_conv/kernel. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (3, 3, 3, 32)

In [2]:
# ==============================
# Fragility Detection Training Script
# ==============================

import os, numpy as np, tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# -----------------------------
# 1️⃣ Paths & Constants
# -----------------------------
DATA_DIR = r"../Datasets/FragilityLevelDetection"
BATCH = 16
IMG = (224,224)


In [3]:
# -----------------------------
# 2️⃣ Data Generators (RGB Fix ✅)
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG,
    color_mode='rgb',           # ✅ Converts grayscale to 3-channel
    batch_size=BATCH,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG,
    color_mode='rgb',           # ✅ Converts grayscale to 3-channel
    batch_size=BATCH,
    class_mode='categorical',
    subset='validation'
)


# -----------------------------
# 3️⃣ Compute Class Weights (for imbalance)
# -----------------------------
labels = train_gen.classes
class_weights = dict(enumerate(
    compute_class_weight('balanced', classes=np.unique(labels), y=labels)
))
print("✅ Class weights:", class_weights)



Found 2829 images belonging to 2 classes.
Found 706 images belonging to 2 classes.
✅ Class weights: {0: np.float64(1.5408496732026145), 1: np.float64(0.7401883830455259)}


In [4]:
batch_x, batch_y = next(train_gen)
print("✅ Batch shape:", batch_x.shape)


✅ Batch shape: (16, 224, 224, 3)


In [5]:
 try:
    base = tf.keras.applications.EfficientNetB0(
        input_shape=(224,224,3),
        include_top=False,
        weights='imagenet'
    )
except ValueError:
    print("⚠️ Falling back to random weights (no pretrained ImageNet).")
    base = tf.keras.applications.EfficientNetB0(
        input_shape=(224,224,3),
        include_top=False,
        weights=None
    )
z


⚠️ Falling back to random weights (no pretrained ImageNet).


In [ ]:
# -----------------------------
# 5️⃣ Custom Classification Head
# -----------------------------
x = base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.002))(x)
x = layers.Dropout(0.5)(x)
out = layers.Dense(train_gen.num_classes, activation='softmax')(x)

model = models.Model(inputs=base.input, outputs=out)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()


In [ ]:
# -----------------------------
# 6️⃣ Callbacks
# -----------------------------
os.makedirs("../Models", exist_ok=True)
cp = ModelCheckpoint("../Models/fragility_best.keras", monitor='val_accuracy', save_best_only=True)
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
rl = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)

# -----------------------------
# 7️⃣ Initial Training (Frozen Base)
# -----------------------------
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    class_weight=class_weights,
    callbacks=[cp, es, rl]
)


In [ ]:
# -----------------------------
# 8️⃣ Fine-Tuning (Unfreeze Last Layers)
# -----------------------------
base.trainable = True
for layer in base.layers[:-50]:  # keep early layers frozen
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_f = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weights,
    callbacks=[cp, es, rl]
)

# -----------------------------
# 9️⃣ Evaluate Final Model
# -----------------------------
val_loss, val_acc = model.evaluate(val_gen)
print(f"\n✅ Final Validation Accuracy: {val_acc*100:.2f}%")


In [ ]:
# -----------------------------
# 🔟 Save Final Model
# -----------------------------
model.save("../models/final_fragility_model.keras")
print("\n💾 Model saved successfully as '../models/final_fragility_model.keras'")